In [2]:
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 MB 7.2 MB/s eta 0:00:0000:0100:01


In [3]:
!pip install torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 575.8 kB/s eta 0:00:00a 0:00:01


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Check if GPU is available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [3]:
# Transformations to apply to the data
transform = transforms.Compose([
    transforms.ToTensor(), # Convert image to PyTorch Tensor
    #transforms.Normalize((0.1307,), (0.3081,)) # Normalize pixel values - std mean and dev
])

# Download and load the training data
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

# Download and load the test data
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=1000, shuffle=False, num_workers=2)


100%|████████████████████████████████████| 9.91M/9.91M [00:01<00:00, 8.62MB/s]
100%|█████████████████████████████████████| 28.9k/28.9k [00:00<00:00, 233kB/s]
100%|████████████████████████████████████| 1.65M/1.65M [00:00<00:00, 2.25MB/s]
100%|████████████████████████████████████| 4.54k/4.54k [00:00<00:00, 5.79MB/s]


In [4]:
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128) # Input layer -> Hidden layer 1
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)   # Hidden layer 1 -> Hidden layer 2
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 10)    # Hidden layer 2 -> Output layer

    def forward(self, x):
        x = self.flatten(x)    # Flatten the image
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)        # Raw scores (logits)
        return x

# Instantiate the model and move it to the appropriate device (GPU or CPU)
model = SimpleMLP().to(device)
print(model)

SimpleMLP(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)


In [5]:
# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001) # Learning rate = 0.001

In [6]:
num_epochs = 10 # Number of times to iterate over the training dataset
training_losses = [] # To store loss values for plotting

print("Starting Training...")
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = data[0].to(device), data[1].to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if (i + 1) % 200 == 0: # Print every 200 mini-batches
            avg_loss = running_loss / 200
            print(f'Epoch [{epoch + 1}/{num_epochs}], Batch [{i + 1}/{len(trainloader)}], Loss: {avg_loss:.4f}')
            training_losses.append({"epoch": epoch + (i+1)/len(trainloader), "loss": avg_loss})
            running_loss = 0.0

print('Finished Training')

Starting Training...
Epoch [1/10], Batch [200/938], Loss: 0.6989
Epoch [1/10], Batch [400/938], Loss: 0.2868
Epoch [1/10], Batch [600/938], Loss: 0.2334
Epoch [1/10], Batch [800/938], Loss: 0.2044
Epoch [2/10], Batch [200/938], Loss: 0.1535
Epoch [2/10], Batch [400/938], Loss: 0.1416
Epoch [2/10], Batch [600/938], Loss: 0.1361
Epoch [2/10], Batch [800/938], Loss: 0.1237
Epoch [3/10], Batch [200/938], Loss: 0.1012
Epoch [3/10], Batch [400/938], Loss: 0.0931
Epoch [3/10], Batch [600/938], Loss: 0.0886
Epoch [3/10], Batch [800/938], Loss: 0.0896
Epoch [4/10], Batch [200/938], Loss: 0.0732
Epoch [4/10], Batch [400/938], Loss: 0.0754
Epoch [4/10], Batch [600/938], Loss: 0.0631
Epoch [4/10], Batch [800/938], Loss: 0.0682
Epoch [5/10], Batch [200/938], Loss: 0.0521
Epoch [5/10], Batch [400/938], Loss: 0.0531
Epoch [5/10], Batch [600/938], Loss: 0.0573
Epoch [5/10], Batch [800/938], Loss: 0.0583
Epoch [6/10], Batch [200/938], Loss: 0.0435
Epoch [6/10], Batch [400/938], Loss: 0.0456
Epoch [6/10

In [7]:
correct = 0
total = 0
# Since we're not training, we don't need to calculate gradients
with torch.no_grad():
    model.eval() # Set model to evaluation mode
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        # Calculate outputs by running images through the network
        outputs = model(images)
        # The class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network on the 10000 test images: {accuracy:.2f} %')


Accuracy of the network on the 10000 test images: 97.50 %
